In [2]:
import pandas as pd

In [9]:
fg_user_known = pd.read_csv('../datas/interim/fg_user_known.csv')
fg_user_unknown = pd.read_csv('../datas/interim/fg_user_unknown.csv')
df = pd.read_csv('../datas/interim/matched_edits_all.csv')

In [8]:
fg_user_known.head()
fg_user_known.drop(['Unnamed: 0'], axis=1)

,before,after,previous_output,type_of_change,category,significance,finegrain_output_raw,after_json_text,username,date,article
0,[[File:1 марта 2014 года в Донецке. У областно...,"--- \n+++ \n@@ -285,7 +285,7 @@\n ===8 April==...","Changed the section title from ""Arrests made d...",section title change,Terminology Manipulation,The change from 'Arrests made due to the prote...,"```json\n{\n ""detected_changes"": [\n {\n ...",==Political prisoners of the Kiev Junta==,2001:690:2100:1E:B1BA:D566:31DD:4E2F,unknown,unknown
1,'''Ukrainian literature''' is [[literature]] w...,No changes,No changes made. \n\nWeaponised,no change,Selective Omission,The lack of changes suggests that the existing...,"```json\n{\n ""detected_changes"": [{""before"": ...",No changes,Alex Bakharev,unknown,unknown
2,#REDIRECT [[History of the Soviet Union (1985-...,"--- \n+++ \n@@ -17,7 +17,7 @@\n 14. [[Ukraine]...","Changed ""The dissolution of the Soviet Union w...",synonym swap,Framing & Emphasis Shifts,This change shifts the focus from the dissolut...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -17,7 +17,7 @@\n 14. [[Ukraine]...",Gary,unknown,unknown
3,The first identifiable groups to populate what...,"--- \n+++ \n@@ -199,7 +199,7 @@\n ===Orange Re...",The change made is the addition of the sentenc...,addition of a sentence,Selective Insertion,The added sentence introduces a fringe claim t...,"```json\n{\n ""detected_changes"": [\n {\n ...","In 2004, Leonid Kuchma announced that he would...",68.39.174.205,unknown,unknown
4,The war between Germany and the Soviet Union d...,"--- \n+++ \n@@ -51,12 +51,12 @@\n \n Much of t...",The change made in this revision is the remova...,section replacement and content addition,Framing & Emphasis Shifts,The edit shifts the narrative from a simple li...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -51,12 +51,12 @@\n \n Much of t...",DMorpheus,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...
508,{{current|date=March 2014}}\n'''Crimea's acces...,"--- \n+++ \n@@ -61,9 +61,11 @@\n | casualties_...",The change made is a rephrasing of the descrip...,rephrasing,Framing & Emphasis Shifts,The edit reframes the narrative of Crimea's an...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -61,9 +61,11 @@\n | casualties_...",Haberstr,unknown,unknown
509,"ALEXANDER II. (1818-1881), emperor of Russia, ...","--- \n+++ \n@@ -81,7 +81,7 @@\n The student ac...",The change made is a **rephrasing** of the sen...,insertion of inappropriate content,Glorification or Vilification,The addition of crude and irrelevant language ...,"```json\n{\n ""detected_changes"": [\n {\n ...","After the last assassination attempt, he mastu...",166.109.0.60,unknown,unknown
510,'''Christmas Eve''' is the day before [[Christ...,"--- \n+++ \n@@ -1,6 +1,53 @@\n {{Other uses}}\...",The change made in this revision is the **addi...,addition of extensive content,Selective Insertion,This edit introduces a significant amount of c...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -1,6 +1,53 @@\n {{Other uses}}\...",184.22.95.32,unknown,unknown
511,#REDIRECT: [[Crimea]],"--- \n+++ \n@@ -49,7 +49,7 @@\n \n In 1783, Cr...","Removed the phrase ""but the vote was boycotted...",deletion,Selective Omission,The removal of the phrase 'but the vote was bo...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -49,7 +49,7 @@\n \n In 1783, Cr...",Citation bot,unknown,unknown


In [8]:
fg_user_unknown.sample()

,Unnamed: 0.1,Unnamed: 0,before,after,previous_output,type_of_change,category,significance,finegrain_output_raw,after_json_text,username,date,article
854,1139,1139,Capital and largest city of the [[Ukraine]] (2...,"--- \n+++ \n@@ -61,7 +61,7 @@\n |blank_info ...",The change made is a rephrasing of the introdu...,rephrasing and contextualization,Terminology Manipulation,This change emphasizes the Ukrainian name 'Kyi...,"```json\n{\n ""detected_changes"": [\n {\n ...","--- \n+++ \n@@ -61,7 +61,7 @@\n |blank_info ...",unknown,unknown,unknown


In [10]:
df.head()

,article,user,date,comment,llm_output,weaponised,ngram,df_index,row_index_matched,fg_row_index,...,clean_before,clean_after,type_of_change_extracted,category_extracted_clean,propaganda_similarity,category_extracted_propaganda_mapped,aligned_before_chunk,aligned_after_chunk,similarity,significance_extracted
0,COVID-19_pandemic_in_Ukraine_analysis,Agathoclea,2020-03-11T20:56:06Z,removed [[Category:2019–20 coronavirus outbrea...,"Changed the category from ""2019–20 coronavirus...",Not Weaponised,annexation of Crimea by,0,15,6,...,A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,rephrasing and addition of descriptive terms,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness",A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,0.925267,The use of terms like 'bloodless' and 'bloody ...
1,History_of_Ukraine_analysis,Icey,2006-05-21T14:09:22Z,/* Further reading */ Disambiguation link repa...,Changed the reference format for Andrew Wilson...,Not Weaponised,A referendum in the,4,384,6,...,A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,rephrasing and addition of descriptive terms,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness",A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,0.925267,The use of terms like 'bloodless' and 'bloody ...
2,History_of_Ukraine_analysis,Irpen,2006-06-06T21:00:08Z,"this whole section doesn't belong here, speara...","Removed a section titled ""Ukraine and Nuclear ...",Not Weaponised,A referendum in the,4,389,6,...,A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,rephrasing and addition of descriptive terms,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness",A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,0.925267,The use of terms like 'bloodless' and 'bloody ...
3,History_of_Ukraine_analysis,Icey,2006-05-21T14:09:22Z,/* Further reading */ Disambiguation link repa...,Changed the reference format for Andrew Wilson...,Not Weaponised,referendum in the largely,4,384,6,...,A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,rephrasing and addition of descriptive terms,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness",A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,0.925267,The use of terms like 'bloodless' and 'bloody ...
4,History_of_Ukraine_analysis,Irpen,2006-06-06T21:00:08Z,"this whole section doesn't belong here, speara...","Removed a section titled ""Ukraine and Nuclear ...",Not Weaponised,referendum in the largely,4,389,6,...,A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,rephrasing and addition of descriptive terms,Terminology Manipulation,0.418396,"Obfuscation, intentional vagueness",A referendum in the largely ethnic Russian Ukr...,A referendum in the largely ethnic Russian Ukr...,0.925267,The use of terms like 'bloodless' and 'bloody ...
